In [18]:
from parse import parse, compile as parse_compile

def parse_input(_input):
    pc = parse_compile('target area: x={x1:d}..{x2:d}, y={y1:d}..{y2:d}')
    return pc.parse(_input)

test = parse_input('target area: x=20..30, y=-10..-5')
prod = parse_input('target area: x=56..76, y=-162..-134')

In [78]:
class TargetArea:
    
    def __init__(self, target):
        self.x1 = min(target['x1'], target['x2'])
        self.x2 = max(target['x1'], target['x2'])
        self.y1 = max(target['y1'], target['y2'])
        self.y2 = min(target['y1'], target['y2'])
        
    def contains(self, point):
        x,y = point    
        return x >= self.x1 and x <= self.x2 and y <= self.y1 and y >= self.y2
    
    def possible(self, point):
        x,y = point
        return x <= self.x2 and y >= self.y2
    
def tests():
    target = TargetArea(test)
    
    assert target.contains((20,-5)) is True
    assert target.contains((19,-5)) is False
    assert target.contains((20,-10)) is True
    assert target.contains((20,-11)) is False
    
    assert target.possible((20,-4)) is True
    assert target.possible((19,-4)) is True
    assert target.possible((30, -4)) is True
    assert target.possible((30, -11)) is False
    print('All tests passed!')
    
tests()

All tests passed!


In [79]:
class Probe:
    
    def __init__(self, velocity, target):
        self.target_reached = False
        self.target_possible = True
        self.target = target
        self.velocity = velocity
        self.position = (0,0)
        self.ymax = 0
        
    def step(self):
        x_velocity, y_velocity = self.velocity
        x,y = self.position
        
        x += x_velocity
        y += y_velocity
        self.position = (x,y)
        self.ymax = max(y, self.ymax)
        
        x_velocity = 0 if x_velocity == 0 else x_velocity - 1 if x_velocity > 0 else x_velocity + 1
        y_velocity -= 1
        self.velocity = (x_velocity, y_velocity)
        
        self.target_reached  = self.target.contains(self.position)
        self.target_possible = self.target.possible(self.position)

In [159]:
class ProbeLauncher:
    
    def __init__(self, target):
        self.target = target
        self.probes = []
        self.generate_probes()
        
    def generate_valid_dx(self):
        x1 = self.target.x1
        x2 = self.target.x2
        valid_dx = []
        for x in range(1, x2 + 1):
            pos = 0
            vel = x
            while True:
                pos += vel
                if pos >= x1 and pos <= x2:
                    valid_dx.append(x)
                    break
                elif pos > x2:
                    break
                vel -= 1
                if vel == 0:
                    break
        return valid_dx
    
    def generate_valid_dy(self):
        y1 = self.target.y1
        y2 = self.target.y2
        valid_dy = []
        for y in range(y2, abs(y2)):
            pos = 0
            vel = y
            while True:
                pos += vel
                if pos <= y1 and pos >= y2:
                    valid_dy.append(y)
                    break
                elif pos < y2:
                    break
                vel -= 1
        return valid_dy
        
    def generate_probes(self):
        for x in self.generate_valid_dx():
            for y in self.generate_valid_dy():
                self.probes.append(Probe((x,y), self.target))
                
    def step(self):
        for probe in self.unresolved_probes():
            probe.step()
                
    def unresolved_probes(self):
        return list(filter(lambda probe: not probe.target_reached and probe.target_possible, self.probes))

# Parts 1 and 2

In [167]:
target = TargetArea(prod)
probe_launcher = ProbeLauncher(target)
while len(probe_launcher.unresolved_probes()) > 0:
    probe_launcher.step()
part_1 = max(probe_launcher.probes, key=lambda p: p.ymax).ymax
part_2 = len(list(filter(lambda p: p.target_reached, probe_launcher.probes)))

print(f"Part 1: {part_1} | Part 2: {part_2}")

Part 1: 13041 | Part 2: 1031
